In [ ]:
import requests

# Recuperiamo l'elenco aggiornato di dataflow ISTAT in formato XML
url_dataflow = "https://esploradati.istat.it/SDMXWS/rest/dataflow"
response = requests.get(url_dataflow)
response.raise_for_status()

# Parsing del contenuto XML
root = ET.fromstring(response.content)

# Namespace con 'xml' incluso per accedere ai titoli in italiano
ns = {
    "structure": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/structure",
    "common": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/common",
    "xml": "http://www.w3.org/XML/1998/namespace"
}

# Estrazione dei dataset
dataflows = []
for df in root.findall(".//structure:Dataflow", ns):
    dataset_id = df.attrib['id']
    name_it = df.find(".//common:Name[@xml:lang='it']", ns)
    name_en = df.find(".//common:Name[@xml:lang='en']", ns)
    name = name_it.text if name_it is not None else (name_en.text if name_en is not None else "")
    dataflows.append({"dataset_id": dataset_id, "title": name})

df = pd.DataFrame(dataflows)

# Filtro per dataset rilevanti a tema economico/sociale
parole_chiave = "coltiv|prodott|produzione|superfici|dop|igp|qualità|famigl|occup|lav|imprese|valore aggiunto|reddito|consum|pil"
df_filtrato = df[df["title"].str.lower().str.contains(parole_chiave, na=False)].reset_index(drop=True)

ModuleNotFoundError: No module named 'ace_tools'

In [13]:
df_filtrato

,dataset_id,title
0,101_1015,Coltivazioni
1,101_1015_DF_DCSP_COLTIVAZIONI_1,Superfici e produzione - dati in complesso
2,101_1015_DF_DCSP_COLTIVAZIONI_2,Superfici e produzione - dati in complesso - prov
3,101_1030,Prodotti agroalimentari di qualità DOP/IGP e STG
4,101_1030_DF_DCSP_DOPIGP_2,Operatori per prodotto
...,...,...
822,EMP,Occupazione
823,IND,Indice della produzione industriale
824,PPI,Prezzi alla produzione
825,SDDS_PLUS_CPI_DF_167_744_1,Prezzi al consumo


In [15]:

import requests
import xml.etree.ElementTree as ET
import pandas as pd

# Scarica l'elenco di tutte le datastructure disponibili da ISTAT SDMX
url = "https://esploradati.istat.it/SDMXWS/rest/datastructure"
response = requests.get(url)
root = ET.fromstring(response.content)

# Namespace XML usati da SDMX
ns = {
    "message": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message",
    "structure": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/structure",
    "common": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/common"
}

# Estrae tutte le strutture (id, agencyID, version, nome IT/EN)
records = []
for dsd in root.findall(".//structure:DataStructure", ns):
    ref_id = dsd.attrib.get("id")
    agency_id = dsd.attrib.get("agencyID")
    version = dsd.attrib.get("version")
    name_it = name_en = ""
    for name in dsd.findall("common:Name", ns):
        if name.attrib.get("{http://www.w3.org/XML/1998/namespace}lang") == "it":
            name_it = name.text
        elif name.attrib.get("{http://www.w3.org/XML/1998/namespace}lang") == "en":
            name_en = name.text
    records.append({
        "id": ref_id,
        "agencyID": agency_id,
        "version": version,
        "name_it": name_it,
        "name_en": name_en
    })



ModuleNotFoundError: No module named 'ace_tools'

In [16]:
df_dsd

,id,agencyID,version,name_it,name_en
0,ECOFIN_DSD,IMF,1.0,,ECOFIN Data Structure Definition
1,BES_TERRIT,IT1,1.1,SIR indicators territorial level,Indicatori SIR a livello territoriale
2,COMP_FRA_IND_MUNICIPAL,IT1,1.0,Indici di fragilità,Fragility indexes
3,DCAR_ATT_NOTAR,IT1,1.0,Atti notarili,Notarial deeds
4,DCAR_CONTECON_ASLAO,IT1,1.0,"Conto economico delle Asl, aziende ospedaliere...","Local health units, hospital companies, univer..."
...,...,...,...,...,...
832,PROVAORDINE,IT1,1.0,PROVAORDINE,PROVAORDINE
833,TERRIT_PROVA_NUOVE_NUTZ,IT1,1.1,,prova
834,TEST,IT1,1.0,SIR indicators territorial level,Indicatori SIR a livello territoriale
835,TEST_DCIS_ABORTISPONTL,IT1,1.0,Aborti spontanei - dimissioni rispetto al luog...,Spontaneous abortions - resignation from the p...


In [14]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

# === URL SDMX da cui scaricare ===
url = "https://sdmx.istat.it/SDMXWS/rest/data/41_983/A.F.001001.KILLINJ"

# === Namespaces XML ===
ns = {
    "generic": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic",
    "message": "http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message"
}

# === Scarica e parse il file XML ===
r = requests.get(url)
root = ET.fromstring(r.content)

# === Estrai osservazioni ===
data = []
for obs in root.findall(".//generic:Series/generic:Obs", ns):
    year = obs.find("generic:ObsDimension", ns).attrib["value"]
    value = obs.find("generic:ObsValue", ns).attrib["value"]
    data.append((int(year), float(value)))

# === In DataFrame ===
df = pd.DataFrame(data, columns=["Anno", "Valore"])
print(df)


    Anno  Valore
0   2001    10.0
1   2002    10.0
2   2003     7.0
3   2004    13.0
4   2005     2.0
5   2006     1.0
6   2007    12.0
7   2008    10.0
8   2009     7.0
9   2010    14.0
10  2011     6.0
11  2012     8.0
12  2013     5.0
13  2014    11.0
14  2015     9.0
15  2016     8.0
16  2017     0.0
17  2018     1.0
18  2019     5.0
19  2020     4.0
20  2021     2.0
21  2022     5.0
22  2023     6.0
